In [ ]:
!pip install crewai crewai_tools google-generativeai python-dotenv


In [ ]:
from google.colab import userdata

api_key = userdata.get("GEMINI_API_KEY")

if api_key is None:
    raise ValueError("❌ Add your GEMINI_API_KEY to Colab → Settings → Secrets")

print("✅ Gemini API key loaded successfully!")


In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from google.genai.errors import ClientError

gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",
    api_key=api_key,
    temperature=0.2,
)


In [ ]:
# Research Agent
csv_research_agent = Agent(
    role="CSV Research Agent",
    goal="Define complete requirements for a Jupyter-based CSV Analyzer tool.",
    backstory="A senior data analyst who knows exactly what matters in data exploration.",
    llm=gemini_llm,
    verbose=True,
)

# Coding Agent
csv_coding_agent = Agent(
    role="CSV Coding Agent",
    goal="Write a full CSVAnalyzer Python class that works perfectly in a notebook.",
    backstory="Expert Python developer who writes clean, readable, reusable code.",
    llm=gemini_llm,
    verbose=True,
)

# Evaluation Agent
csv_evaluation_agent = Agent(
    role="CSV Evaluation Agent",
    goal="Review the CSVAnalyzer code for correctness, clarity, and robustness.",
    backstory="A strict reviewer focused on real-world reliability.",
    llm=gemini_llm,
    verbose=True,
)


In [ ]:
csv_research_task = Task(
    description=(
        "Turn the user goal into a clean specification.\n"
        "Goal: Build a notebook-based CSV Analyzer tool.\n\n"
        "Required features:\n"
        "- load CSV\n"
        "- overview: rows, cols, info(), head(), tail()\n"
        "- numeric stats: describe()\n"
        "- missing values summary\n"
        "- duplicate detection\n"
        "- correlation matrix (heatmap)\n"
        "- error handling\n"
        "- dummy CSV auto-creation\n"
        "- usage example\n"
        "Return a numbered REQUIREMENTS list."
    ),
    agent=csv_research_agent,
    expected_output="A numbered list of requirements for the CSV Analyzer."
)

csv_coding_task = Task(
    description=(
        "Write FULL PYTHON CODE implementing:\n"
        "- A CSVAnalyzer class with methods:\n"
        "    load, overview, numeric stats, missing values, duplicates,\n"
        "    correlation matrix\n"
        "- Auto-create dummy CSV (data.csv) with missing values + duplicates\n"
        "- Provide a usage example at the bottom\n"
        "- Provide simple tests at the end\n"
        "Code must be valid Python ONLY."
    ),
    agent=csv_coding_agent,
    expected_output="A complete Python script for CSVAnalyzer.",
    context=[csv_research_task]
)

csv_evaluation_task = Task(
    description=(
        "Review the generated CSVAnalyzer code.\n"
        "Return:\n"
        "- WHAT_WORKS\n"
        "- ISSUES\n"
        "- SUGGESTIONS\n"
    ),
    agent=csv_evaluation_agent,
    expected_output="WHAT_WORKS, ISSUES, SUGGESTIONS",
    context=[csv_research_task, csv_coding_task]
)


In [ ]:
csv_crew = Crew(
    agents=[csv_research_agent, csv_coding_agent, csv_evaluation_agent],
    tasks=[csv_research_task, csv_coding_task, csv_evaluation_task],
    process=Process.sequential,
    verbose=True,
)

try:
    final_output = csv_crew.kickoff()
    print("\n==== FINAL CSV ANALYZER RESULT ====\n")
    print(final_output)

except ClientError as e:
    print("❌ Gemini API Error:", e)

except Exception as e:
    print("❌ Unexpected Error:", e)
